In [ ]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

f1 = mss.add(Fix((0,0,0)))

mA = mss.add(Mass(1, (1,0,-0.5)))
mB = mss.add(Mass(1, (2,0,-1.5)))

L1 = 1.0
L2 = 1.0

# --- ADD CONSTRAINTS ---
dc1 = DistanceConstraint(f1, mA, L1)
dc2 = DistanceConstraint(mA, mB, L2)

mss.addDistanceConstraint(dc1)
mss.addDistanceConstraint(dc2)



In [ ]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []

for c in mss.constraints:
    # connector 1
    if c.c1.type == 1:                      # FIX
        pA = mss.fixes[c.c1.nr].pos
    else:                                   # MASS
        pA = mss.masses[c.c1.nr].pos

    # connector 2
    if c.c2.type == 1:
        pB = mss.fixes[c.c2.nr].pos
    else:
        pB = mss.masses[c.c2.nr].pos

    springpos.append([pA, pB])


springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [ ]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

In [ ]:
from time import sleep
for i in range(200):
    mss.simulate (0.02, 1)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)